**State :**
1. **Schema of the graph**
    1. TypedDict
    2. Pydantic
2. **Reducer functions**
    1. Pre-defined (operator.add, add_messages, etcs.)
    2. Custom functions


<p align="center">
  <img src="langgraph_state_definition2.gif" width="800">
</p>

#### TypedDict

In [ ]:
from typing_extensions import TypedDict

In [ ]:
class TypedState(TypedDict):
    name: str
    age: int

In [ ]:
TypedState(name="Rahul", age=30)

In [ ]:
TypedState(name="Rahul", age="30 years", years = 5)

#### Pydantic

1. Separate python library for data validation
2. Automatic type checking and conversion.
3. Supports default values, optional fields, and validators.

In [ ]:
from pydantic import BaseModel

In [ ]:
class PydanticState(BaseModel):
    name: str
    age: int

In [ ]:
PydanticState(name="Rahul", age=30)

In [ ]:
PydanticState(name="Rahul", age="30")

In [ ]:
PydanticState(name="Rahul", age="30 years")

In [ ]:
TypedState(name="Rahul", age=-1)

In [ ]:
PydanticState(name="Rahul", age=-1)

In [ ]:
from pydantic import field_validator

In [ ]:
class PydanticState(BaseModel):
    name: str
    age: int

    @field_validator("age")
    def validate_age(cls, value):
        if value < 0:
            raise ValueError("Age must be a positive integer")
        return value

In [ ]:
PydanticState(name="Rahul", age=-1)

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image # type: ignore

In [ ]:
def node_a(state : PydanticState) -> PydanticState:
    state.age += 1
    return state

In [ ]:
builder = StateGraph(PydanticState)
builder.add_node("A", node_a)
builder.add_edge(START, "A")
builder.add_edge("A", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"name": "Rahul", "age": 30}) # type: ignore

In [ ]:
graph.invoke({"name": "Rahul", "age": -2}) # type: ignore

### Multiple Schemas

All graph nodes usually share a single schema, reading and writing to the same state.

Sometimes, more control is needed:

        1. Internal nodes can pass info not required in the graph’s input/output.

        2. Graphs can use different input/output schemas; e.g., output may include only a single relevant key.

#### 1. Private

In [ ]:
class OverallState(TypedDict):
    name: str
    age: int

class PrivateState(TypedDict):
    salary: int

In [ ]:
def node_a(state : OverallState) -> PrivateState:
    print("node A state: ", state)
    return {"salary" : 10000}

def node_b(state : PrivateState) -> OverallState:
    print("node B state: ", state)
    return state # type: ignore

In [ ]:
builder = StateGraph(OverallState)
builder.add_node("A", node_a)
builder.add_node("B", node_b)
builder.add_edge(START, "A")
builder.add_edge("A", "B")
builder.add_edge("B", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"name": "Rahul", "age": 30})

#### 2. Input and Output Schema

In [ ]:
class OverallState(TypedDict):
    name: str
    age: int
    info: str

class InputState(TypedDict):
    age: int

class OutputState(TypedDict):
    info: str

In [ ]:
def node_a(state : InputState):
    print("node A - input state: ", state)
    return {"name" : "Rahul", "age" : state["age"] + 1} # type: ignore

def node_b(state : OverallState) -> OutputState:
    print("node B - overall state: ", state)
    return {"info": f"{state['name']} is {state['age']} years old."}

In [ ]:
builder = StateGraph(OverallState, input_schema=InputState, output_schema=OutputState)
builder.add_node("A", node_a)
builder.add_node("B", node_b)
builder.add_edge(START, "A")
builder.add_edge("A", "B")
builder.add_edge("B", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"age" : 30})